In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Configuring Devices

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

Using: cuda


Collecting Data




In [3]:
data = load_breast_cancer()
x = data.data
y = data.target

Train / Test Split

In [4]:
# train and test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [5]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [6]:
print(x.shape), print(x_train.shape), print(x_test.shape)

(569, 30)
(455, 30)
(114, 30)


(None, None, None)

Preprocessing

In [7]:
# standardizing data (with standardscalar)
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [8]:
# converting numpy arrays to tensors for GPU processing
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

Building Neural Network!

In [9]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [10]:
# defining hyperparameters
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1     # 0 or 1
lr = 0.001
num_epochs = 100

In [11]:
# initializing model
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [12]:
# defining loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Training!

In [13]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(x_train)
  loss = criterion(outputs, y_train.view(-1, 1))

  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum()
    accuracy = correct / y_train.size(0)

  if (epoch + 1) % 10 == 0:
    print(f"Epoch #{epoch + 1}/{num_epochs}, Loss = {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")


Epoch #10/100, Loss = 0.4927, Accuracy: 89.23%
Epoch #20/100, Loss = 0.3871, Accuracy: 91.43%
Epoch #30/100, Loss = 0.3082, Accuracy: 92.75%
Epoch #40/100, Loss = 0.2499, Accuracy: 93.63%
Epoch #50/100, Loss = 0.2068, Accuracy: 94.95%
Epoch #60/100, Loss = 0.1752, Accuracy: 96.04%
Epoch #70/100, Loss = 0.1516, Accuracy: 97.36%
Epoch #80/100, Loss = 0.1337, Accuracy: 97.80%
Epoch #90/100, Loss = 0.1199, Accuracy: 97.80%
Epoch #100/100, Loss = 0.1089, Accuracy: 98.02%


Model Evaluation

In [14]:
# training set evaluation
def evaluate_model(data):
  if data == 'train':
    x = x_train
    y = y_train
  else:
    x = x_test
    y = y_test

  model.eval()
  with torch.no_grad():
    outputs = model(x)
    predicted = outputs.round()
    correct = (predicted == y.view(-1, 1)).float().sum()
    accuracy = correct / y.size(0)
    print(f"{data} Accuracy: {accuracy.item() * 100:.2f}%")

In [15]:
evaluate_model("train")

train Accuracy: 98.02%


In [16]:
evaluate_model("test")

test Accuracy: 96.49%
